# Paired statements demo

This notebook demonstrates how to distil the *Matters of Concern* section of scraped PFD reports into discrete concerns using the `ConcernParser`.

The example uses the sample data under `../scripts/data/responses`, where reports are linked to responses via the `url`/`parent_url` columns.

In [1]:
from dotenv import load_dotenv
import os
import pandas as pd

from pfd_toolkit import ConcernParser, LLM


In [2]:
# Load OpenAI API key from local environment
load_dotenv('api.env')
openai_api_key = os.getenv('OPENAI_API_KEY')

# Initialise LLM client
llm_client = LLM(
    api_key=openai_api_key,
    max_workers=30,
    model='gpt-4.1-mini',
    seed=123,
    temperature=0,
    timeout=60,
)


In [3]:
reports_path = '../scripts/data/responses/reports.csv'
responses_path = '../scripts/data/responses/responses.csv'

reports_df = pd.read_csv(reports_path).drop(columns=['Unnamed: 0'], errors='ignore')
responses_df = pd.read_csv(responses_path).drop(columns=['Unnamed: 0'], errors='ignore')

reports_df.head()


,url,id,date,coroner,area,receiver,investigation,circumstances,concerns
0,https://www.judiciary.uk/prevention-of-future-...,2024-0621,2024-11-13,J. Andrews,"West Sussex, Brighton and Hove",Chief Executive Officer South East Coast Ambul...,On 4 October 2023 I commenced an investigation...,Joel Colk called 999 at 21:46 on 2 October 202...,The Court heard that when a call is made to 99...
1,https://www.judiciary.uk/prevention-of-future-...,2024-0623,2024-11-13,J. Gittins,North Wales (East and Central),Kingkabs Wheatsheaf Garage,On 16 October 2022 I commenced an investigatio...,"On the 15 December 2022, the deceased was coll...",Oral testimony was given by a representative o...
2,https://www.judiciary.uk/prevention-of-future-...,2024-0636,2024-11-12,D. Stewart,Suffolk,"Chief Executive, West Suffolk NHS Foundation T...",On 20 July 2023 I commenced an investigation t...,The circumstances of the death are recorded in...,Evidence received at Inquest included NICE gui...
3,https://www.judiciary.uk/prevention-of-future-...,2024-0619,2024-11-12,S. Fox,Avon,Royal College of Pathologists; Royal College o...,On 20 April 2023 I commenced an investigation ...,Lisa developed a rare but serious condition of...,(1) Blood was taken for liver function tests (...
4,https://www.judiciary.uk/prevention-of-future-...,2024-0618,2024-11-12,L. Lee,Coventry,NHS England; The Chief Executive Officer of Un...,"On 9 January 2024, an investigation was commen...","In March 2022, John underwent a kidney transpl...",Concern 1: Contacting specialist centres\nNon-...


In [4]:
parser = ConcernParser(llm=llm_client, reports=reports_df, responses=responses_df)
concerns = parser.parse_concerns(output="object")
concerns


Sending requests to the LLM: 100%|██████████| 28/28 [00:06<00:00,  4.47it/s]


ConcernResults(reports=[ReportConcerns(url='https://www.judiciary.uk/prevention-of-future-death-reports/joel-colk-prevention-of-future-deaths-report/', report_id='2024-0621', recipients=['Chief Executive Officer South East Coast Ambulance Service NHS Foundation Trust', 'NHS England & NHS Improvement'], concern_set=ConcernSet(concerns=[ConcernItem(concern='The NHS Pathways system used for 999 calls categorises all overdoses identically without differentiating by type, severity, substance, amount ingested, timing, or patient weight, potentially leading to inappropriate response prioritisation.'), ConcernItem(concern='The Pathways system does not account for the time-sensitive nature of treatment for [REDACTED] ingestion, failing to assign a higher urgency category despite the rapid onset of cardiac arrest associated with this chemical.'), ConcernItem(concern='South East Coast Ambulance Service NHS Foundation Trust ambulances do not carry Methylene Blue, the antidote for [REDACTED] ingest

In [5]:
# JSON-ready list
concerns.as_list()[:2]


[{'url': 'https://www.judiciary.uk/prevention-of-future-death-reports/joel-colk-prevention-of-future-deaths-report/',
  'id': '2024-0621',
  'receiver': ['Chief Executive Officer South East Coast Ambulance Service NHS Foundation Trust',
   'NHS England & NHS Improvement'],
  'responses': ["Thank you for your Report to Prevent Future Deaths (hereafter “Report”) dated 13 November 2024 concerning the death of Joel Phillip Colk on 2 October 2023. In advance of responding to the specific concerns raised in your Report, I would like to express my deep condolences to Joel’s family and loved ones. NHS England are keen to assure the family and the Coroner that the concerns raised about Joel’s care have been listened to and reflected upon.\n\nIn your Report, you raised that the NHS Pathways system does not differentiate between types of, severity of or the drugs / chemicals reported as being the cause of an overdose. The system also does not take into account the amount reported as ingested, the

In [6]:
# Flat DataFrame view
concerns.as_df().head()


,url,id,receiver,concern
0,https://www.judiciary.uk/prevention-of-future-...,2024-0621,[Chief Executive Officer South East Coast Ambu...,The NHS Pathways system used for 999 calls cat...
1,https://www.judiciary.uk/prevention-of-future-...,2024-0621,[Chief Executive Officer South East Coast Ambu...,The Pathways system does not account for the t...
2,https://www.judiciary.uk/prevention-of-future-...,2024-0621,[Chief Executive Officer South East Coast Ambu...,South East Coast Ambulance Service NHS Foundat...
3,https://www.judiciary.uk/prevention-of-future-...,2024-0623,[Kingkabs Wheatsheaf Garage],The taxi driver indicated a willingness to lea...
4,https://www.judiciary.uk/prevention-of-future-...,2024-0623,[Kingkabs Wheatsheaf Garage],Kingkabs did not contact the police in the inc...


In [7]:
parser.count("all")


{'reports': 28,
 'responses': 48,
 'mean_responses_per_report': 1.7142857142857142,
 'median_responses_per_report': 1.0,
 'max_responses_per_report': 6,
 'concerns': 105}